# **Create a Technology Type Column for FERC Steam Table**
---------------

## Setup

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Standard libraries
import logging
import sys
import os
import pathlib
import random

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa
#import pickle

# Local libraries
import pudl
from pudl.analysis.fill_ferc1_fuel_gaps import *
from pudl.analysis.flag_ferc1_totals import *

# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 75
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [4]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq='AS'
)

In [5]:
value_cols_no_cap = [
    'net_generation_mwh','avg_num_employees',
    'capex_land', 'capex_equipment', 'capex_structures', 'capex_total', 'asset_retirement_cost',
    'opex_operations', 'opex_fuel', 'opex_coolants', 'opex_steam', 'opex_steam_other', 'opex_transfer', 
    'opex_electric', 'opex_misc_power', 'opex_rents', 'opex_allowances', 'opex_engineering', 'opex_structures', 
    'opex_boiler', 'opex_plants', 'opex_misc_steam', 'opex_production_total'
]

value_cols = value_cols_no_cap + ['capacity_mw']

test_view = ['report_year', 'utility_name_ferc1', 'plant_name_ferc1', 'plant_id_pudl', 
             'plant_id_ferc1', 'primary_fuel', 'plant_type', 
             'tech_desc', 'tech_desc_flag','record_id', 'capacity_mw']

total_view = test_view + ['total_type']

ferc_merge_cols = ['report_year', 'utility_id_ferc1', 'plant_name_ferc1']
eia_merge_cols = ['report_date', 'plant_id_pudl', 'generator_id']

In [6]:
# Establish table connections
steam = pudl_out.plants_steam_ferc1().assign(primary_fuel=np.nan)
fbp = pudl_out.fbp_ferc1()
fbp_small = fbp[ferc_merge_cols + ['primary_fuel_by_mmbtu']] #, 'primary_fuel_by_cost']]
eia = pudl_out.gens_eia860()#.assign(report_year=lambda x: x.report_date.dt.year)
small_plants = pudl_out.plants_small_ferc1()#.dropna(subset=['plant_name_ferc1'])
eia_one_plant = eia[eia['plant_name_eia'].notna()].drop_duplicates(subset='plant_name_eia').reset_index()
eia_one_tech = eia.groupby(['plant_id_pudl']).filter(lambda x: len(x.technology_description.dropna().unique()) == 1)
small_plants_one_plant = small_plants[small_plants['plant_name_ferc1'].notna()].drop_duplicates(subset='plant_name_ferc1').reset_index()
fuel = pudl_out.fuel_ferc1()

In [14]:
#flag_steam_totals(steam)

 - flagging specific totals
 - adding manual totals
 - backfilling totals by capacity


## **1. Generate Tech Descriptions**

In [7]:
# Add fuel type and update plant type field in FERC
fuel_plant_df = (
    steam.pipe(impute_fuel_type, pudl_out)
    .pipe(impute_plant_type)
)

 - adding fuel types
  * filling fuels with obvious names
    27240 / 29270 rows left unfilled
  * filling in primary fuel by mmbtu
    4040 / 29270 rows left unfilled
  * filling in eia plants with one reported fuel
    2734 / 29273 rows left unfilled
  * filling in primary fuel by cost
    2458 / 29273 rows left unfilled
  * filling in raw ferc1 fuels
    2299 / 29273 rows left unfilled
  * filling in ferc plants with one fuel
    1472 / 29273 rows left unfilled
  * filling in pudl plants with one fuel
    1343 / 29273 rows left unfilled
  * filling in manually mapped fuels
    1254 / 29273 rows left unfilled
  * front and backfilling values with the same ferc1 id
    996 / 29273 rows left unfilled
  * flipping single fuel outliers for plant_id_ferc1
    996 / 29273 rows left unfilled
  * flipping multiple fuel outliers for groups under 7
    996 / 29273 rows left unfilled
  * flipping single outliers by capacity
 - adding plant types
  * filling plants with obvious names
    798 / 2

In [8]:
# Merge EIA technology description with FERC
tech_df = impute_tech_desc(fuel_plant_df, eia)

 - adding tech types
  * merging single-tech EIA technology_description with FERC
    25528 / 28555 rows left unfilled
  * backfilling EIA technology_description by year if no new units installed
    15171 / 28555 rows left unfilled
  * combining primary_fuel and plant_type columns
    728 / 28555 rows left unfilled
  * making uniform tech description col


**List of plants to double check:**

In [8]:
# - 241
# - 1282 Waterside - not a lot of information on what this is or when it stopped...
# - 2259 hunterstown - con-ed petro unclear...capacities don't match what I found online at alll
# - 2260 Mountain - another weird con-ed petro plant....no idea
# - 1157 Joppa - confusing subunits
# - 1149 hopewell there is another hopewell in eia (3335) but the capacity from 1149 matches....:/
# - 45 bellmeade - some of it's right some not
# - 176 elizabeth river - plant type just wrong
# - 1132 gibson city - some weird outlier fuels and plant types
# - 488 remington - some solar plants thrown in at the end! need to fix
# - 5930 - airport - seems that the primary fuel is right (rather than petro from eia)
# - 121 coit - unknown valeus causing a problem
# - 1656 --> False is the result of a NA in the construction_year field which should probably be ignored?


# - any that are waste
# --- 15 Altavista - went from coal to biomass in 2013
# --- 646 wilmarth - waste
# --- 548 southampton - went from coal to biomass in 2013
# --- 517 rothschild - waste
# - any that are marked as coal but not coal_steam
# - any with a weird fuel flip flop that wasn't accounted for in the fixes
# - any plants with more than one technology type

-----------

### Test accuracy of fuel + plant type

In [392]:
# grab rows that were already present in eia and make a few tweaks for the analysis
tech_df_copy = (
    tech_df.loc[tech_df['tech_desc_flag'].isin([
        'direct from eia860', 'backfill from eia year', 'backfill from other year'])].copy()
    .assign(
        primary_fuel=lambda x: x.primary_fuel.replace({'oil': 'petroleum'}, regex=True),
        similar_plant=np.nan,
        similar_fuel=np.nan))

no_null_plant = tech_df_copy['plant_type'].notna()
no_null_fuel = tech_df_copy['primary_fuel'].notna()
no_null_tech = tech_df_copy['tech_desc_no_map'].notna()

mini_plant = (
    tech_df_copy[no_null_plant & no_null_tech].copy()
    .assign(similar_plant=lambda x: x.apply(lambda x: x.plant_type in x.tech_desc_no_map, axis=1))
)

mini_fuel = (
    tech_df_copy[no_null_fuel & no_null_tech].copy()
    .assign(similar_fuel=lambda x: x.apply(lambda x: x.primary_fuel in x.tech_desc_no_map, axis=1))
)

tech_df_copy.update(mini_plant, overwrite=True)
tech_df_copy.update(mini_fuel, overwrite=True)

#test2.loc[(test2['similar_plant'].notna()) & (test2['similar_fuel'].notna()), 'similar'] =  
tech_df_copy['similar'] = tech_df_copy['similar_plant'] & tech_df_copy['similar_fuel']

test = tech_df_copy[no_null_plant & no_null_fuel & no_null_tech].copy()
print(len(test), ' -- total rows with an eia description')
print(len(test[test['similar']]), ' -- rows where fuel + plant type matches the eia description')

13263  -- total rows with an eia description
10472  -- rows where fuel + plant type matches the eia description


In [49]:
n1 = test['similar_fuel'] == False
n2 = test['similar_plant'] == False

print('pudl plants with non-matching fuel+plant and eia technology descriptions:')
print('*note that many of these are waste')
test[n1 & n2][test_view].plant_id_pudl.astype('int').unique()

pudl plants with non-matching fuel+plant and eia technology descriptions:
*note that many of these are waste


array([ 242, 1282,  364, 2259, 2260,  283,  647, 1135, 1156,   15, 1148,
        549, 2628,  518,  489, 5931, 1525,    5])

### Look at plants that still have more than one technology type

In [ ]:
more_than_one_fuel = tech_df.groupby(['plant_id_pudl']).filter(lambda x: len(x.tech_desc.dropna().unique()) > 1)
len(more_than_one_fuel.plant_id_pudl.unique())
more_than_one_fuel.plant_id_pudl.unique()

In [26]:
# Look for ferc1 ids that have more than one fuel associated with them
more_than_one_fuel = test2.groupby(['plant_id_ferc1']).filter(lambda x: len(x.tech_type.dropna().unique()) > 1)
len(more_than_one_fuel.plant_id_ferc1.unique())
more_than_one_fuel.plant_id_ferc1.unique().astype('int')

array([   2,    3,    5,   15,   44,   59,   64,   65,   73,   74,   75,
         76,   77,   78,   79,   80,   91,  136,  142,  175,  189,  190,
        202,  209,  210,  214,  216,  223,  246,  247,  255,  295,  296,
        327,  329,  330,  347,  356,  359,  363,  365,  372,  397,  398,
        423,  478,  481,  507,  540,  553,  554,  565,  575,  576,  578,
        601,  632,  674,  681,  708,  711,  712,  713,  729,  747,  750,
        752,  845,  897,  901,  903,  904,  908,  911,  925,  934,  940,
        946,  947,  954,  958,  978, 1009, 1020, 1034, 1056, 1057, 1062,
       1078, 1092, 1093, 1094, 1098, 1099, 1104, 1107, 1129, 1139, 1157,
       1159, 1160, 1162, 1181, 1191, 1195, 1208, 1215, 1218, 1224, 1235,
       1294, 1318, 1332, 1342, 1355, 1356, 1456, 1479, 1484, 1494, 1503,
       1532, 1543, 1545, 1554, 1556, 1557, 1605, 1618, 1619, 1623, 1659,
       1712, 1723, 1828, 1886])

In [46]:
# Shows that of the plant_id_ferc1 values with more than one fuel over time, there are none that start
# and end with the same fuel type (meaning, hopefully, there are no pockets in the middle.)
more_than_one_fuel.sort_values(['report_year'])
same_first_last = more_than_one_fuel.groupby(['plant_id_ferc1'])['tech_type'].agg(lambda x: x.max==x.min)
more_than_one_fuel['same_first_last'] = more_than_one_fuel.plant_id_ferc1.map(same_first_last)
more_than_one_fuel[more_than_one_fuel['same_first_last']][test_view + ['same_first_last']].plant_id_ferc1.unique().astype('int')

array([], dtype=int64)

In [66]:
dd = test2[(test2['tech_type'].str.contains('coal')) & (test2['tech_type'].str.contains('steam')==False)][test_view + ['tech_type']]
len(dd.plant_id_pudl.astype('int').unique())

91

## **2. Flag Totals**

In [10]:
flagged_tots_df = flag_steam_totals(tech_df)

 - flagging specific totals
 - adding manual totals
 - backfilling totals by capacity


#### Total Values:

* **plant total:** pudl plant totals where the plant is owned by more than one utility.
* **utility owned total:** all of the assets owned by a utility
* **utility owned plant total:** a utility's owned portion of a pudl plant
* **utility owned plant total steam:** all of the utility's steam assets within a given pudl plant
* **utility owned plant total nuclear:** all of the utility's nuclear assets within a given pudl plant
* **utility owned subtotal:** the sum of several units within a pudl plant owned by one utility
* **utility owned plant extra:** any extra amounts that are associated with a plant
* **utility owned extra:** any extra amounts that are associated with all a utility's assets
* **unit total:** the sum of a co-owned unit (sub-pudl plant id)
---------

### Test total groups

In [25]:
no_tots = flagged_tots_df[flagged_tots_df['total_type'].isna()].copy()

In [ ]:
no_tots['tech_desc']

In [12]:
flagged_tots_df

In [ ]:
# plant total 
# -- compare total values if multiple utilities reporting these totals
# -- use total value from one utility for another (if needed)


In [11]:
pudl_id_list = flagged_tots_df[flagged_tots_df['total_type']=='plant total']['plant_id_pudl'].unique().tolist()
test_df = flagged_tots_df[flagged_tots_df['plant_id_pudl'].isin(pudl_id_list)]

In [12]:
test_df.plant_id_pudl.unique()

array([ 288,  250,   16,  451, 1208,  612,  289,  317,  296, 1087,  383,
        337,  308, 1665, 1165,  124,  530,  344,  611,  531,  168,  104,
        653])

In [83]:
one_plant = test_df[test_df['plant_id_pudl']==383]
test = one_plant.sort_values(['report_year'])[test_view + ['primary_fuel_flag', 'total_type']]

In [ ]:
#flagged_tots_df.query(f"plant_id_pudl=={random.choice(outlier_plants)}")[test_view]

### Find more fuel rows to flip

In [24]:
#flagged_tots_df.query("plant_id_pudl==277")[test_view]

In [34]:
aa = flagged_tots_df.assign(primary_fuel=lambda x: x.primary_fuel.fillna('unknown'))
#show_year_outliers(aa).sort_values('unique_fuel_groups').query("new_year==False")

## **3. Aggregate to Utility-Tech Level**

In [139]:
def agg_n_flag(flag_df, agg_col):
    """Aggregate specified column and create flag column based on total rows.
    
    """
    # For each report_year, utility_id_ferc1, plant_id_pudl, tech_desc group...
    
    # If all non-ttl rows have values and there are more than 0 just add 'em up
    if (flag_df.loc[flag_df['total_type'].isna()][agg_col].notna().all()) & (len(flag_df.loc[flag_df['total_type'].isna()]) > 0): 
        flag = 'actual values provided'#None
        agg_value = flag_df.loc[flag_df['total_type'].isna()][agg_col].sum()
        return [agg_value, flag]
    
    # If there aren't any total rows, just take the sum cuz that's all you can do
    elif flag_df['total_type'].isna().all():
        flag = 'no total rows' #None
        agg_value = flag_df[agg_col].sum()
        return [agg_value, flag]
    
    # Now there ARE total rows and there ARE value rows with NA:
    # If the total rows are all NA for the agg col, just take the sum cuz that's all you can do
    elif flag_df[flag_df['total_type'].notna()][agg_col].isna().all():
        flag = 'totals are NA'
        agg_value = flag_df[agg_col].sum()
        return [agg_value, flag]
    
    # If there are only total rows there's probably an issue with plant/fuel type mapping
    elif flag_df['total_type'].notna().all():
        flag = 'all rows only totals...'
        agg_value = flag_df[agg_col].sum() # FIX 
        return [agg_value, flag]
    
    # If non-total value cols are all NA (and by default there are total rows with values):....FIX
    elif flag_df[flag_df['total_type'].isna()][agg_col].isna().all():
        flag = flag_df[flag_df[agg_col].notna()]['total_type'].unique()[0] # this might be bad...
        agg_value = flag_df[flag_df['total_type'].notna()][agg_col].sum()
        return [agg_value, flag]

    # If some non-total value cols are NA but not all (and by default there are total rows with values):
    else:
        flag = 'some values are NA but not all'
        agg_value = flag_df[agg_col].sum() # FIX
        return [agg_value, flag]

In [140]:
def run_the_thing(df, agg_col):
    print(f'starting aggregation for {agg_col}')
    plant_group = df.groupby(['report_year', 'utility_id_ferc1', 'plant_id_pudl', 'tech_desc'])
    col_agg_series = plant_group.apply(lambda x: agg_n_flag(x, agg_col))
    col_agg_df = pd.DataFrame(col_agg_series).reset_index()
    col_agg_df[[agg_col, f'{agg_col}_flag']] = pd.DataFrame(col_agg_df[0].tolist(), index=col_agg_df.index)
    col_agg_df = col_agg_df.drop(columns=[0])
    
    util_agg_df = (
        col_agg_df.groupby(['report_year', 'utility_id_ferc1', 'tech_desc'])
        .agg({agg_col: 'sum', f'{agg_col}_flag': lambda x: ', '.join(list(x.dropna().unique()))})
    )
    return util_agg_df

In [142]:
def run_the_whole_thing(df):
    agg_df = pd.DataFrame(columns=['report_year', 'utility_id_ferc1', 'tech_desc'])
    for col in value_cols:
        one_col_df = run_the_thing(df, col)
        #df_list.append(one_col_df)
        agg_df = pd.merge(agg_df, one_col_df, on=['report_year', 'utility_id_ferc1', 'tech_desc'], how='outer')
    return agg_df

In [151]:
whole_enchilada = run_the_whole_thing(flagged_tots_df)

starting aggregation for net_generation_mwh
starting aggregation for avg_num_employees
starting aggregation for capex_land
starting aggregation for capex_equipment
starting aggregation for capex_structures
starting aggregation for capex_total
starting aggregation for asset_retirement_cost
starting aggregation for opex_operations
starting aggregation for opex_fuel
starting aggregation for opex_coolants
starting aggregation for opex_steam
starting aggregation for opex_steam_other
starting aggregation for opex_transfer
starting aggregation for opex_electric
starting aggregation for opex_misc_power
starting aggregation for opex_rents
starting aggregation for opex_allowances
starting aggregation for opex_engineering
starting aggregation for opex_structures
starting aggregation for opex_boiler
starting aggregation for opex_plants
starting aggregation for opex_misc_steam
starting aggregation for opex_production_total
starting aggregation for capacity_mw


In [146]:
whole_enchilada.to_excel('/Users/aesharpe/Desktop/full_tech_aggregation.xlsx')

# **Test Small Generators Table**
-------------

In [ ]:
# Steps:
# - clean and normalize rows
# - name obvious rows

In [103]:
#! pip install fuzzywuzzy
from fuzzywuzzy import process, fuzz

/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# see if you can work capacity into this or use Christina's matching mechanism that only checks within utility id

eia_one_tech_ = eia_one_tech.dropna(subset=['technology_description'])

plant_name_eia = []
similarity = []
for i in small_plants_one_plant.plant_name_ferc1:
        ratio = process.extract( i, eia_one_tech_.plant_name_eia, limit=1)
        plant_name_eia.append(ratio[0][0])
        similarity.append(ratio[0][1])
small_plants_one_plant['plant_name_eia'] = pd.Series(plant_name_eia)
small_plants_one_plant['similarity'] = pd.Series(similarity)

In [109]:
print(len(nada))
print(len(nada[nada['plant_id_pudl'].notna()]))

7765
1825


In [156]:
# flag total rows and remove them for now

small_plants['total_row'] = False
small_plants.loc[small_plants['plant_name_ferc1'].str.contains('total'), 'total_row'] = True
small_plants = small_plants[small_plants['total_row']==False].copy()

In [157]:
# fix fuel types
fix_fuel_types = {
    'coal': ['coal'],
    'oil': ['diesel', 'natural gas', 'diesel oil', 'oil', '#2 oil', '# 2 oil', 'no. 2', '#2', '#2 diesel', 'diesel-low s',
            '#1 diesel', '#2 ls diesel', '#1 ls diesel', '#1 uls diesel', '#2 uls diesel', 'diesel-ultra low s',
            'diesle-low s', 'diesel/', 'no. 2 oil', 'no. 6 oil', 'diesel fuel', 'fuel oil', 'bunker oil', 'no. 2 diesel'],
    'gas': ['natural gas', 'gas', 'propane', 'methane', 'methane gas', 'nat gas', 'nat. gas', 'ng'],
    'solar': ['solar'],
    'wind': ['wind'], 
    'waste': ['waste heat', 'landfill gas'],
    'hydro': ['hydro', 'water', 'run-of-rr', 'run-of-rv', 'run of river', 'run-of-river'],
    'unknown': ['na', 'n/a', '--', '#6', '---------', 'a', 'a, b', 'a,b', 'pil', 'steam', '-', 'none', 'n.a.', '"', 'fossil', '0',
                None, '77509', '120976'], 
    'unclear': ['coal and gas', 'coal & gas', 'gas / oil', 'gas/oil', 'coal/oil', 'oil,n.gas', 'oil,nat.gas', 'oil-gas', 'oil/gas', 
                'diesel-low s or gas', 'gas-oil', 'gas - oil', 'coal/gas', 'rdf, gas', 'gas & oil', 'oil, nat .gas', 'oil, nat. gas',
                'waste heat/gas', 'gas, fuel oil']
}

# map "clean" fuels onto table
small_plants['clean_fuel'] = small_plants.fuel_type
for fuel in fix_fuel_types:
    small_plants['clean_fuel'] = small_plants.clean_fuel.replace(fix_fuel_types[fuel], fuel)

In [158]:
# map obvious names

def no_fake_names(fuel, name_exceptions):
    no_fake_names = ~small_plants['plant_name_ferc1'].isin(name_exceptions)
    good_names = small_plants['plant_name_ferc1'].str.contains(fuel)
    return good_names, no_fake_names

solar_name_exceptions = ['solar centaur gas turbine', 'solar centaur gas turbine (#13)']
solar_names, no_fake_solar = no_fake_names('solar', solar_name_exceptions)

gas_name_exceptions = ['gaston shoals (5)', 'gaston shoals (5)', 'gaston shoals - project 2332',
                       'las vegas solar', 'wellford landfill gas to energy plant',
                       'blackfoot landfill gas generation facility']
gas_names, no_fake_gas = no_fake_names('gas', gas_name_exceptions)

wind_name_exceptions = ['new windsor rd, hinsdale ma', 'windsor rd, savoy ma', 'lake winds energy park',
                        'cross winds energy park', 'bison wind energy center'] # sc-etwind10-sol?
wind_names, no_fake_wind = no_fake_names('wind', wind_name_exceptions)

small_plants.loc[solar_names & no_fake_solar_names, 'clean_fuel'] = 'solar'
small_plants.loc[gas_names & no_fake_gas_names, 'clean_fuel'] = 'gas'
small_plants.loc[small_plants['plant_name_ferc1'].str.contains('hydro'), 'clean_fuel'] = 'hydro'
small_plants.loc[small_plants['plant_name_ferc1'].str.contains('nuclear'), 'clean_fuel'] = 'nuclear'
small_plants.loc[small_plants['plant_name_ferc1'].str.contains('diesel'), 'clean_fuel'] = 'oil'
small_plants.loc[small_plants['plant_name_ferc1'].str.contains('oil'), 'clean_fuel'] = 'oil'

In [159]:
# Jankily assign pudl id to those that already have one

# For now, removing any duplicates from the pudl map...
ferc_eia_map = pd.read_excel("/Users/aesharpe/Desktop/mapping_eia923_ferc1.xlsx", sheet_name='plants_output')
ferc_eia_map = ferc_eia_map.drop_duplicates(subset=['plant_name_ferc1', 'utility_id_ferc1'], keep=False)

small_plants_pudl_id = pd.merge(small_plants, ferc_eia_map, on=['plant_name_ferc1', 'utility_id_ferc1'], how='left')
small_plants_pudl_id['plant_id_pudl'] = small_plants_pudl_id.plant_id_pudl.astype('Int64')

print('rows with pudl id:', len(small_plants_pudl_id[small_plants_pudl_id['plant_id_pudl'].notna()]))
print('rows without pudl id:', len(small_plants_pudl_id[small_plants_pudl_id['plant_id_pudl'].isna()]))

print('plants with pudl id:', len(small_plants_pudl_id[small_plants_pudl_id['plant_id_pudl'].notna()].plant_id_pudl.unique()))

rows with pudl id: 3132
rows without pudl id: 11232
plants with pudl id: 659


In [160]:
# Merge with EIA technology description

bb_eia = (
    eia_one_tech[['plant_id_pudl', 'plant_name_eia', 'technology_description']].copy()
    .drop_duplicates()
    .dropna(subset=['technology_description']))

plants_with_eia_desc = [plant for plant in bb_eia.plant_id_pudl.unique().dropna() if plant in small_plants_pudl_id.plant_id_pudl.unique().dropna()]
plants_without_eia_desc = [plant for plant in small_plants_pudl_id.plant_id_pudl.dropna() if plant not in plants_with_eia_desc]

print('plants with a possible eia tech description:', len(plants_with_eia_desc))
print('plants without an eia tech description:', len(plants_without_eia_desc))


plants with a possible eia tech description: 307
plants without an eia tech description: 1487


In [162]:
# Add the tech description without accounting for year
small_plants_tech_desc = pd.merge(small_plants_pudl_id, bb_eia, on=['plant_id_pudl'], how='left')

# There are much fewer rows that got filled in likely because many of the technolgy descriptions in EIA were logged as NA
print('rows with non NA tech description:', len(small_plants_tech_desc[small_plants_tech_desc['technology_description'].notna()]))
print('rows with NA tech desc:', len(small_plants_tech_desc[small_plants_tech_desc['technology_description'].isna()]))

rows with non NA tech description: 1673
rows with NA tech desc: 12719


In [163]:
sptd = small_plants_tech_desc.copy()

In [164]:
no_tech = sptd['technology_description'].isna()
no_fuel = sptd['clean_fuel'].isin(['unknown', 'unclear'])

print(len(nada := sptd[no_tech & no_fuel]))
print(len(sptd))

6701
14392


In [165]:
len(nada[nada['plant_id_pudl'].notna()])

761